In [1]:
from dataset import BPI2012Dataset
from parameters.dataset import BPI2012Parameters
from utils.save import get_json_dict
from parameters.training import TrainingParameters, LossParameters, OptimizerParameters
from parameters.model import LSTMPredNextAmountSpecificModelParameters
from model import LSTMPredNextAmountSpecificModel
from controller import TrainingController

In [2]:
bpi_2012_parms = BPI2012Parameters()

In [3]:
get_json_dict(bpi_2012_parms)

{'file_path': './data/event_logs/BPI_Challenge_2012.xes',
 'preprocessed_folder_path': './data/preprocessed/BPI_Challenge_2012_with_resource',
 'include_types': ['A', 'O', 'W'],
 'include_complete_only': True}

In [4]:
dataset = BPI2012Dataset(bpi_2012_parms)


| Preprocessed data loaded successfully: ./data/preprocessed/BPI_Challenge_2012_with_resource/AOW_CompleteOnly 


In [5]:
model_params = LSTMPredNextAmountSpecificModelParameters()

In [6]:
get_json_dict(model_params)

{'activity_embedding_dim': 32,
 'resource_embedding_dim': 128,
 'lstm_hidden': 64,
 'dense_dim': 64,
 'dropout': 0.1}

In [7]:
model = LSTMPredNextAmountSpecificModel(
    activity_vocab = dataset.activity_vocab,
    resource_vocab = dataset.resource_vocab,
    parameters = model_params
)

In [8]:
train_params = TrainingParameters(
    stop_epoch=20,
)

In [9]:
get_json_dict(train_params)

{'stop_epoch': 20,
 'batch_size': 128,
 'verbose_freq': 250,
 'run_validation_freq': 80,
 'train_test_split_portion': [0.8, 0.1],
 'random_seed': 12345}

In [10]:
loss_params = LossParameters()

In [11]:
get_json_dict(loss_params)

{'loss': 'CrossEntropy'}

In [12]:
optim_params = OptimizerParameters()

In [13]:
get_json_dict(optim_params)

{'optimizer': 'Adam',
 'learning_rate': 0.005,
 'l2': 1e-10,
 'lr_scheduler': 'ExponentialDecay',
 'lr_step_scheduler_step': 800,
 'lr_setp_scheduler_gamma': 0.8,
 'lr_exp_decay_scheduler_step': 1000,
 'lr_exp_decay_scheduler_rate': 0.96,
 'lr_exp_decay_scheduler_staircase': True,
 'SGD_momentum': 0.9}

In [14]:
controller = TrainingController(
    dataset,
    model,
    train_params,
    optim_params,
    loss_params,
)


| Running on /job:localhost/replica:0/task:0/device:CPU:0  


In [15]:
controller.model_info()

Model: "LSTMPredNextAmountSpecificModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  832       
_________________________________________________________________
embedding_1 (Embedding)      multiple                  9216      
_________________________________________________________________
lstm (LSTM)                  multiple                  24832     
_________________________________________________________________
lstm_1 (LSTM)                multiple                  33024     
_________________________________________________________________
lstm_2 (LSTM)                multiple                  49408     
_________________________________________________________________
lstm_3 (LSTM)                multiple                  33024     
_________________________________________________________________
sequential (Sequential)      (1, 64

In [16]:
# tensorboard --logdir logs/gradient_tape --host localhost --port 8088
test_accuracy = controller.train()


| Training records in logs/gradient_tape/20210701-202017 

| Total epochs: 20 

| Total steps: 1640 

| Start epoch 0 

| Evaluation result | Loss [0.7198] | Accuracy [0.0730]  

| Start epoch 1 

| Evaluation result | Loss [0.6927] | Accuracy [0.0829]  

| Start epoch 2 

| Evaluation result | Loss [0.4801] | Accuracy [0.3648]  

| Start epoch 3 

| Evaluation result | Loss [0.2513] | Accuracy [0.5823]  

| Start epoch 4 

| Evaluation result | Loss [0.1442] | Accuracy [0.8147]  

| Start epoch 5 

| Evaluation result | Loss [0.1249] | Accuracy [0.8228]  

| Start epoch 6 

| Evaluation result | Loss [0.1223] | Accuracy [0.8241]  

| Start epoch 7 

| Evaluation result | Loss [0.1212] | Accuracy [0.8250]  

| Start epoch 8 

| Evaluation result | Loss [0.1218] | Accuracy [0.8231]  

| Start epoch 9 

| Evaluation result | Loss [0.1201] | Accuracy [0.8240]  

| Start epoch 10 

| Evaluation result | Loss [0.1206] | Accuracy [0.8237]  

| Start epoch 11 

| Evaluation result | Loss [0.

In [17]:
# test_accuracy = controller.perform_eval_on_testset()

In [18]:
save_folder_path = model.get_folder_path("bpi_2012_amount_specific_training.ipynb", test_accuracy , dataset.info_str())

In [19]:
controller.save_parameters(save_folder_path)
model.save(save_folder_path)


| Parameters saved successfully to: ./SavedModels/0.8175_LSTMPredNextAmountSpecificModel_AOW_CompleteOnly_2021-07-01 20:45:16.353467  

| Vocabs saved successfully to: ./SavedModels/0.8175_LSTMPredNextAmountSpecificModel_AOW_CompleteOnly_2021-07-01 20:45:16.353467  

| Model saved successfully to: ./SavedModels/0.8175_LSTMPredNextAmountSpecificModel_AOW_CompleteOnly_2021-07-01 20:45:16.353467  


In [20]:
# controller.load_parameters(savet_folder_path)
# model.load(save_folder_path)